In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk


from googletrans import Translator
translator = Translator()

import warnings
warnings.filterwarnings('ignore')

from textblob import TextBlob

In [2]:
reviews_df = pd.read_csv('googleplaystore_reviews2.csv')
reviews_df.sample(10)
#print(len(reviews_df))

,App,Review
133820,aCalendar+ Calendar & Tasks,I have used this for years because of the ease...
82529,Transformers Rescue Bots: Disaster Dash,I like all the controls and graphics but I wis...
65118,Helix Jump,Great game. Highly recommended. Had a similar ...
131510,IFTTT,Brilliant. But beware the rabbit hole you will...
25065,Lumosity: #1 Brain Games & Cognitive Training App,I was starting to like the app. So I paid. Sil...
94441,Meetup,Great app for finding things to do in big citi...
49990,Weight Loss Tracker - RecStyle,I used this app to track my wieght and body fa...
46947,Relax with Andrew Johnson Lite,"Excellent app Soothing voice, great way to rel..."
114350,Cricbuzz - Live Cricket Scores & News,Best app for cricket fans. The UI is simple ye...
117153,高德地图,Why does this app ask for phone permission?


In [3]:
import re
def clean(text):
        '''
        Utility function to clean text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \\
                                    |(\w+:\/\/\S+)", " ", text).split())

reviews_df['Review'] = reviews_df['Review'].apply(lambda x: str(x))
reviews_df['Review'] = reviews_df['Review'].apply(lambda x: clean(x))

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop = stop + ['app', 'ap', 'application', 'browser', 'website', 'websites', 'chrome', 'click', 'web', 'ip', 'address',
            'files', 'android', 'browse', 'service', 'use', 'one', 'download']
print(stop)

reviews_df['Review'] = reviews_df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
grouped = reviews_df.groupby('App')
reviews_df = reviews_df.groupby('App').filter(lambda x: len(x) >= 30)
print(len(grouped.groups))

sentiments_polarity = []
sentiments_subjectivity = []  
translated_reviews = []
sentiments_category =[]
for key,group in grouped:
    
    if len(group) < 30:
        #ignore apps having less than 30 reviews
        continue
    else:
        for review in group.Review:
            try:
                translated_review = translator.translate(review).text
                sentiment = TextBlob(translated_review)
                sentiment_polarity = sentiment.polarity
                if sentiment_polarity == 0.0:
                    sentiment_category = 'Neutral'
                elif sentiment_polarity > 0.0:
                    sentiment_category = 'Positive'
                else:
                    sentiment_category = 'Negative'
                sentiment_subjectivity = sentiment.subjectivity
            except:
                translated_review = np.nan
                sentiment_polarity = np.nan
                sentiment_subjectivity = np.nan
                sentiment_category = np.nan

            #print(sentiment_polarity, sentiment_subjectivity)
            translated_reviews.append(translated_review)
            sentiments_polarity.append(sentiment_polarity)
            sentiments_subjectivity.append(sentiment_subjectivity)
            sentiments_category.append(sentiment_category)
            #print(review + ' *********' + translated_review)

        #print('=' * 50)


reviews_df['Translated_Review'] = translated_reviews
reviews_df['Sentiment'] = sentiments_category
reviews_df['Sentiment_Polarity'] = pd.Series(sentiments_polarity).values
reviews_df['Sentiment_Subjectivity'] = pd.Series(sentiments_subjectivity).values

reviews_df['Translated_Review'] = reviews_df['Translated_Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))

print(reviews_df.columns)

2711


In [ ]:
print(reviews_df.shape)
print(reviews_df.isna().sum())
reviews_df.dropna().size

In [ ]:
reviews_df.to_csv('googleplaystore_sentiment_analysis.csv')